In [ ]:
#!/usr/bin/env python3
#coding=utf-8
import time
from Arm_Lib import Arm_Device
# Create robot arm object
Arm = Arm_Device()
time.sleep(.1)
for i in range(3):
    Arm.Arm_RGB_set(255, 255, 255)
    time.sleep(.25)
    Arm.Arm_RGB_set(0,0, 0)
    time.sleep(.25)
time.sleep(1)

In [ ]:
Arm.Arm_serial_servo_write6(90, 90, 90, 90, 90, 90, 2000)

In [ ]:
Arm.Arm_serial_servo_write6(90, 130, 0, 0, 90, 90, 2000)

In [ ]:
import numpy as np

def get_colors(img):
    color_map = {
        "red": [[], []],
        "yellow": [[], []],
        "green": [[], []],
        "blue": [[], []],
    }
    H = {}
    S = {}
    color_name={}
    img = cv2.resize(img, (80, 60), )
    # Convert color image to HSV
    HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Take out the H, S, and V values of each row and column in turn and put them into the container.
    for i in range(0, 80):
        for j in range(0, 60):
            H[(j, i)] = HSV[j, i][0]
            S[(j, i)] = HSV[j, i][1]
            
    precision = 4
    
    for point, hue in H.items():
        
        if hue >= 0 and hue <= 10:
            color_map["red"][0].append(point)
            color_map["red"][1].append(S[point])
        elif hue >= 26 and hue <= 34:
            color_map["yellow"][0].append(point)
        elif hue >= 35 and hue <= 100:
            color_map["green"][0].append(point)
        elif hue >= 100 and hue <= 224:
            color_map["blue"][0].append(point)
        
    print(color_map)
    
    color_map["red"] = (avg_red_x, avg_red_y)
    
    return color_map["red"]

In [ ]:
import cv2
import ipywidgets.widgets as widgets
import threading
image_widget = widgets.Image(format='jpeg', width=600, height=500) #Set up the camera display component
display(image_widget) #Show camera components


#bgr8 to jpeg format
#image is 640x480
import enum
import cv2
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

def monitor():
    image = cv2.VideoCapture(1) #open camera
    if not image.isOpened():
        print("couldn't open the camera")
        return
    print("running")
    Arm.Arm_RGB_set(0, 255, 0)
    while True:
        ret, frame = image.read()
        try:
            red = get_colors(frame)
            #print(red)
            cv2.circle(frame, (round(red[0] * 8), round(red[1] * 8)), 3, (0, 0, 0), -1)
            image_widget.value = bgr8_to_jpeg(frame)
            Arm.Arm_RGB_set(0, 255, 0)
            time.sleep(1)
            return
        except Exception as e:
            Arm.Arm_RGB_set(255, 0, 0)
            print(e)
            
        time.sleep(1)

In [ ]:
camera_thread = threading.Thread(target=monitor)
camera_thread.start()